In [ ]:
import torch
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d nibinv23/iam-handwriting-word-database

100% 1.10G/1.10G [00:34<00:00, 34.7MB/s]
100% 1.10G/1.10G [00:34<00:00, 34.7MB/s]


In [ ]:
!unzip -q iam-handwriting-word-database.zip

In [ ]:
import os
img_paths=[]
for i in os.listdir('iam_words/words'):
  for j in os.listdir('iam_words/words/'+i):
    for x in os.listdir('iam_words/words/'+i+'/'+j):
      img_paths.append('iam_words/words/'+i+'/'+j+'/'+x)
len(img_paths)

115320

In [ ]:
imgs_labels={}
with open('words_new.txt','r') as labels_txt:
  lines=labels_txt.readlines()
  for i in range(18,len(lines)):
    if lines[i].split(' ')[1].strip()!='ok' or len(lines[i].split(' ')[-1].strip())<=1:
      continue
    img_name=lines[i].strip().split(' ')[0].strip()
    label=lines[i].split(' ')[-1].strip()
    imgs_labels[img_name]=label
    if i==20:
      print('`'+img_name+'`','`'+label+'`')

len(imgs_labels)

`a01-000u-00-02` `to`


33273

In [ ]:
vocab=[chr(a) for a in range(ord('a'),ord('z')+1)]+[chr(a) for a in range(ord('A'),ord('Z')+1)]
for i in imgs_labels:
  for x in imgs_labels[i]:
    if x not in vocab:
      vocab.append(x)
print(vocab)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '.', '-', "'", '1', '9', '5', '8', '3', '4', '0', ',', '2', '7', '6', '/', '*', '?', '"']


In [ ]:
!mkdir preproc_img

In [ ]:
import cv2
import numpy as np
def img_reshape(img,name):
  shape=(32,128)
  target = np.ones(shape)*255
  H, W = shape
  h, w = img.shape
  fx = H/h
  fy = W/w
  f = min(fx, fy)
  _h = int(h*f)
  _w = int(w*f)
  _img = cv2.resize(img, (_w,_h))
  target[:_h,:_w] = _img
  cv2.imwrite(name,target)
i=0
for im in img_paths:
  img_name=im.split('/')[-1].strip().split('.')[0]
  if img_name in imgs_labels:
    img=cv2.imread(im)
    try:
      if len(img.shape)==3:
        img=cv2.imread(im,0)
      img_reshape(img,'preproc_img/'+imgs_labels[img_name]+'@IAM'+str(i)+'.png')
      i+=1
    except:
      print(im)
len(os.listdir('preproc_img'))

iam_words/words/a01/a01-117/a01-117-05-02.png


33263

In [ ]:
!wget https://www.dropbox.com/s/l2ul3upj7dkv4ou/synthetic-data.zip

--2023-07-12 00:11:41--  https://www.dropbox.com/s/l2ul3upj7dkv4ou/synthetic-data.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/l2ul3upj7dkv4ou/synthetic-data.zip [following]
--2023-07-12 00:11:42--  https://www.dropbox.com/s/raw/l2ul3upj7dkv4ou/synthetic-data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc94bd024abcfa4f4e04f690997b.dl.dropboxusercontent.com/cd/0/inline/B_vWHwpUph_-fPJfB69fhOzpUQ5aOUK0LCAZ_AOrR2G0ukTuqydftOZUcp36MXtF-kfb0ZBhvZxTCPyHDb1F85DyhxrqJC9ivdOmO0IrlxqTosNmpyjtworGAZOBDZJCgfIkwFEGZ_F1oYyIcvLG3iZcWs21Drs4N3yNmu51KelrkA/file# [following]
--2023-07-12 00:11:42--  https://uc94bd024abcfa4f4e04f690997b.dl.dropboxusercontent.com/cd/0/inline/B_vWHwpUph_-fPJfB69fhOzpUQ5aOUK0LCAZ_AOrR2G0ukTuqydftOZUcp3

In [ ]:
!unzip -q synthetic-data.zip

In [ ]:
import os
words=[a.strip().split('@')[0] for a in os.listdir('synthetic-data')]
chars=[]
max_len=0
for word in words:
  max_len=len(word) if len(word)>max_len else max_len
  chars.extend([ch for ch in word if ch not in chars])
print(chars)
print(max_len)

['s', 'm', 'a', 'l', 'l', 'w', 'i', 'n', 'h', 'u', 'b', 'd', 't', 'e', 'c', 'e', 'r', 'f', 'o', 'o', 'T', 'V', 'p', 'z', 'g', 'y', 'v', 'k', 'x', 'j', 'q', 'C', 'R', 'I', 'A', 'P', 'M', 'D']
14


In [ ]:
import cv2
import numpy as np
def img_reshape(img,name):
  shape=(32,128)
  target = np.ones(shape)*255
  H, W = shape
  h, w = img.shape
  fx = H/h
  fy = W/w
  f = min(fx, fy)
  _h = int(h*f)
  _w = int(w*f)
  _img = cv2.resize(img, (_w,_h))
  target[:_h,:_w] = _img
  cv2.imwrite(name,target)
for im in os.listdir('synthetic-data'):
  img=cv2.imread('synthetic-data/'+im,0)
  img_reshape(img,'preproc_img/'+im)

In [ ]:
!pip install -q editdistance

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import cv2
import editdistance
import random
class dataset(Dataset):
  def __init__(self,data,timestep=32,img_size=(32,128)):
    super().__init__()
    self.data=data
    self.timestep=timestep
    self.img_size=img_size
    self.vocap=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '.', '-', "'", '1', '9', '5', '8', '3', '4', '0', ',', '2', '7', '6', '/', '*', '?', '"']
    self.char_code={x:i+1 for i,x in enumerate(self.vocap)}
    self.char_code.update({'-':0})
    self.code_char={i+1:x for i,x in enumerate(self.vocap)}
    self.code_char.update({0:'-'})
  def pad(self,code,max_len):
    padded_item = code + [0]*(max_len-len(code))
    return padded_item
  def sample(self):
    return self[random.randint(0,len(self))]
  def get_image_label(self,image_name):
    return image_name.strip().split('@')[0]
  def get_label_code(self,label):
    return [self.char_code[x] for x in label]
  def __len__(self):
    return len(self.data)
  def __getitem__(self, index):
    item=self.data[index]
    image=cv2.imread('preproc_img/'+item,0)
    label=self.get_image_label(item)
    return image,label
  def collate_fn(self, batch):
    images, labels, label_lengths, label_vectors, input_lengths = [], [], [], [], []
    for image, label in batch:
        images.append(torch.Tensor(image)[None,None])
        label_lengths.append(len(label))
        labels.append(label)
        label_code=self.get_label_code(label)
        padded=self.pad(label_code,self.timestep)
        label_vectors.append(padded)
        input_lengths.append(self.timestep)
    images = torch.cat(images).float().to(device)
    label_lengths = torch.Tensor(label_lengths).long().to(device)
    label_vectors = torch.Tensor(label_vectors).long().to(device)
    input_lengths = torch.Tensor(input_lengths).long().to(device)
    return images, labels, label_lengths,label_vectors,  input_lengths
  def wer(self, preds, labels):
      c = 0
      for p, l in zip(preds, labels):
          c += p.lower().strip() != l.lower().strip()
      return round(c/len(preds), 4)
  def cer(self, preds, labels):
      c, d = [], []
      for p, l in zip(preds, labels):
          c.append(editdistance.eval(p, l) / len(l))
      return round(np.mean(c), 4)
  def decode(self, pred):
      decoded = ""
      last = ""
      pred = pred.cpu().detach().numpy()
      for i in range(len(pred)):
          k = np.argmax(pred[i])
          if k > 0 and self.code_char[k] != last:
              last = self.code_char[k]
              decoded = decoded + last
          elif k > 0 and self.code_char[k] == last:
              continue
          else:
              last = ""
      return decoded.replace(" ","")
  def evaluate(self, model, ims, labels, lower=False):
      model.eval()
      preds = model(ims).permute(1,0,2)
      preds = [self.decode(pred) for pred in preds]
      return {'char-error-rate': self.cer(preds, labels),
              'word-error-rate': self.wer(preds, labels),
              'char-accuracy' : 1 - self.cer(preds, labels),
              'word-accuracy' : 1 - self.wer(preds, labels)}


In [ ]:
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
batch=64
data=os.listdir('preproc_img')
trn,val=train_test_split(data,test_size=0.2,random_state=22)
print(len(data))
print(len(trn))
print(len(val))

58395
46716
11679


In [ ]:
import random
import shutil
paths= os.listdir('preproc_img')
for i in range(5):
  pat=paths[random.randint(0,len(paths))]
  shutil.copy('preproc_img/'+pat,pat)

In [ ]:
import os
import cv2
import torchvision.transforms as transforms
from PIL import Image,ImageOps
import numpy as np
rotate_transform =transforms.RandomRotation(degrees=5)
affine_transform=transforms.RandomAffine(degrees=5, translate=(0.01, 0.01), shear=10)
gaussblur_transform=transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))

trans=[rotate_transform,affine_transform,gaussblur_transform]
aug_trn=[]
aug_val=[]
def zoom_out(path,pad,aug_list,ind):
    img = Image.open('preproc_img/'+path)
    padding = pad
    width, height = img.size
    new_width = width + 2 * padding
    new_height = height + 2 * padding
    bordered_img = ImageOps.expand(img, border=padding, fill='white')
    img_zoom_out = bordered_img.resize((128, 32))
    try:
      img_zoom_out.save('preproc_img/'+path.split('@')[0]+'@zoom'+str(pad)+str(ind)+path.split('@')[1], format=path.split('.')[-1])
    except:
      print(path)
    aug_list.append(path.split('@')[0]+'@zoom'+str(pad)+str(ind)+path.split('@')[1])
for path in trn:
    zoom_out(path,25,aug_trn,1)
    zoom_out(path,10,aug_trn,2)
for path in val:
    zoom_out(path,25,aug_val,3)
    zoom_out(path,10,aug_val,4)


In [ ]:
trn+=aug_trn
val+=aug_val
print(len(os.listdir('preproc_img')))
print(len(trn))
print(len(val))

175185
140148
35037


In [ ]:
trn_dataset=dataset(trn)
val_dataset=dataset(val)
trn_loader=DataLoader(trn_dataset,collate_fn=trn_dataset.collate_fn,batch_size=batch,drop_last=True,shuffle=True)
val_loader=DataLoader(val_dataset,collate_fn=val_dataset.collate_fn,batch_size=batch,drop_last=True)
print(len(trn_loader))
print(len(val_loader))

2189
547


In [ ]:
print(len(vocab))

70


In [ ]:
import torch.nn as nn
class OCRmodel(nn.Module):
  def __init__(self,ks=3,st=1,pad=1,pool=2, drop=0.2,vocab_len=70):
    super().__init__()
    self.model=nn.Sequential(
        nn.Conv2d(1, 128, kernel_size=ks, stride=st, padding=pad),
        nn.BatchNorm2d(128, momentum=0.3),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(pool),
        nn.Dropout2d(drop),
        nn.Conv2d(128, 128, kernel_size=ks, stride=st, padding=pad),
        nn.BatchNorm2d(128, momentum=0.3),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(pool),
        nn.Dropout2d(drop),
        nn.Conv2d(128, 256, kernel_size=ks, stride=st, padding=pad),
        nn.BatchNorm2d(256, momentum=0.3),
        nn.ReLU(inplace=True),
        nn.MaxPool2d((4,2)),
        nn.Dropout2d(drop)
    )
    self.rnn=nn.Sequential(
        nn.LSTM(256, 256, num_layers=2, dropout=0.2, bidirectional=True)
    )
    self.classification = nn.Sequential(
        nn.Linear(512, vocab_len+1),
        nn.LogSoftmax(-1),
    )
  def forward(self,x):
    x=self.model(x)
    x=x.reshape(-1, 256, 32)
    x=x.permute(2,0,1)
    x,ls=self.rnn(x)
    x=self.classification(x)
    return x
def ctc(log_probs, target, input_lengths, target_lengths, blank=0):
    loss = nn.CTCLoss(blank=blank, zero_infinity=True)
    ctc_loss = loss(log_probs, target, input_lengths, target_lengths)
    return ctc_loss

In [ ]:
!pip install -q torch_summary
!pip install -q torch_snippets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 36.4 MB/s eta 0:0

In [ ]:
from torchsummary import summary
from torch_snippets import Report
model=OCRmodel()
model.to(device)
summary(model, torch.zeros((1,1,32,128)).to(device))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 256, 2, 16]          --
|    └─Conv2d: 2-1                       [-1, 128, 32, 128]        1,280
|    └─BatchNorm2d: 2-2                  [-1, 128, 32, 128]        256
|    └─ReLU: 2-3                         [-1, 128, 32, 128]        --
|    └─MaxPool2d: 2-4                    [-1, 128, 16, 64]         --
|    └─Dropout2d: 2-5                    [-1, 128, 16, 64]         --
|    └─Conv2d: 2-6                       [-1, 128, 16, 64]         147,584
|    └─BatchNorm2d: 2-7                  [-1, 128, 16, 64]         256
|    └─ReLU: 2-8                         [-1, 128, 16, 64]         --
|    └─MaxPool2d: 2-9                    [-1, 128, 8, 32]          --
|    └─Dropout2d: 2-10                   [-1, 128, 8, 32]          --
|    └─Conv2d: 2-11                      [-1, 256, 8, 32]          295,168
|    └─BatchNorm2d: 2-12                 [-1, 256, 8, 32]          512

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 256, 2, 16]          --
|    └─Conv2d: 2-1                       [-1, 128, 32, 128]        1,280
|    └─BatchNorm2d: 2-2                  [-1, 128, 32, 128]        256
|    └─ReLU: 2-3                         [-1, 128, 32, 128]        --
|    └─MaxPool2d: 2-4                    [-1, 128, 16, 64]         --
|    └─Dropout2d: 2-5                    [-1, 128, 16, 64]         --
|    └─Conv2d: 2-6                       [-1, 128, 16, 64]         147,584
|    └─BatchNorm2d: 2-7                  [-1, 128, 16, 64]         256
|    └─ReLU: 2-8                         [-1, 128, 16, 64]         --
|    └─MaxPool2d: 2-9                    [-1, 128, 8, 32]          --
|    └─Dropout2d: 2-10                   [-1, 128, 8, 32]          --
|    └─Conv2d: 2-11                      [-1, 256, 8, 32]          295,168
|    └─BatchNorm2d: 2-12                 [-1, 256, 8, 32]          512

In [ ]:
from typing import ValuesView
from torch.autograd.variable import Variable
import shutil
early_stop_patience=0
ebochs=100
lr=0.003
optimizer=torch.optim.AdamW(model.parameters(),lr)
log=Report(ebochs)
N = len(trn_loader)
V= len(val_loader)
best_val_acc=0
for eboch in range(ebochs):
  for ix,data in enumerate(trn_loader):
    model.train()
    pos = eboch+(ix+1)/N
    imgs,labels, label_len,targets, input_len = data
    optimizer.zero_grad()
    preds = model(imgs)
    loss=ctc(preds, targets, input_len, label_len)
    loss.backward()
    optimizer.step()
    eval_res=trn_dataset.evaluate(model,imgs,labels)
    log.record(pos=pos, train_loss=loss, train_char_acc=eval_res['char-accuracy'],
               train_word_acc=eval_res['word-accuracy'],end='\r')

  val_acc=0
  with torch.no_grad():
    for ix,data in enumerate(val_loader):
      model.eval()
      pos = eboch+(ix+1)/V
      imgs,labels, label_len,targets, input_len = data
      preds = model(imgs)
      loss=ctc(preds, targets, input_len, label_len)
      eval_res=val_dataset.evaluate(model,imgs,labels)
      log.record(pos=pos, validation_loss=loss, validation_char_acc=eval_res['char-accuracy'],
               validation_word_acc=eval_res['word-accuracy'], end='\r')
      val_acc+=eval_res['word-accuracy']
    if val_acc>best_val_acc:
      early_stop_patience=0
      best_val_acc=val_acc
      torch.save(model.state_dict(),'best.pt')
      print('saving best weights -- epoch {}'.format(eboch))
      print()
    else:
      early_stop_patience+=1
      if early_stop_patience==30:
        print('Early stopping \nsaving last weights...')
        torch.save(model.state_dict(),'last.pt')
    log.report_avgs(eboch+1)
    print()
    for jx in range(5):
          img, label = val_dataset.sample()
          _img = torch.Tensor(img).float().to(device)[None][None]
          pred = model(_img)[:,0,:]
          pred = val_dataset.decode(pred)
          print(f'Pred: `{pred}` :: Truth: `{label}`')
    imahm=cv2.imread('/content/img.jpg',0)
    _imahm=torch.Tensor(imahm).float().to(device)[None,None]
    pred = model(_imahm)[:,0,:]
    pred = val_dataset.decode(pred)
    print("test img: "+pred)
    if pred=="fire":
      torch.save(model.state_dict(),'real_time.pt')
    print()
  torch.save(log,'log.log')


EPOCH: 46.000  validation_loss: 0.203  validation_char_acc: 0.953  validation_word_acc: 0.844  (21163.99s - 24844.68s remaining)saving best weights -- epoch 45

EPOCH: 46.000  train_loss: 0.128  validation_loss: 0.294  validation_char_acc: 0.927  validation_word_acc: 0.782  train_char_acc: 0.987  train_word_acc: 0.936  (21164.22s - 24844.95s remaining)

Pred: `Sany` :: Truth: `Barry`
Pred: `away` :: Truth: `away`
Pred: `if` :: Truth: `if`
Pred: `to` :: Truth: `to`
Pred: `on` :: Truth: `on`
test img: rixe

EPOCH: 47.000  train_loss: 0.123  validation_loss: 0.307  validation_char_acc: 0.924  validation_word_acc: 0.778  train_char_acc: 0.987  train_word_acc: 0.937  (21363.62s - 24090.90s remaining)

Pred: `this` :: Truth: `this`
Pred: `today` :: Truth: `today`
Pred: `firmge` :: Truth: `through`
Pred: `says` :: Truth: `says`
Pred: `Wed` :: Truth: `West`
test img: pie

EPOCH: 48.000  validation_loss: 0.243  validation_char_acc: 0.939  validation_word_acc: 0.844  (21561.39s - 23358.17s remai